# Technical test for the data science internship @Hyperlex

In [20]:
import os
import pandas as pd

rootdir = 'dataset'
json_files = [file for _,_,files in os.walk(rootdir) for file in files if file.endswith('json')]      